In [ ]:
#1. 설치
!pip install transformers datasets accelerate --quiet

#2. 라이브러리 로딩
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset
import json

#3. 아주 가벼운 한국어 모델 (KoGPT2 base)
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = AutoModelForCausalLM.from_pretrained(model_name)

#4. 훈련용 최소 데이터셋 (코드 내 정의)
examples = [
  {"instruction": "안녕?", "input": "", "output": "안녕하세요! 만나서 반가워요."},
  {"instruction": "이름이 뭐야?", "input": "", "output": "저는 인공지능 챗봇이에요."},
  {"instruction": "날씨 어때?", "input": "", "output": "오늘은 맑고 따뜻한 날씨예요."},
  {"instruction": "너는 뭐 할 수 있어?", "input": "", "output": "저는 질문에 대답하고 정보를 제공할 수 있어요."},
  {"instruction": "한국의 수도는 어디야?", "input": "", "output": "한국의 수도는 서울이에요."},
  {"instruction": "사과는 무슨 색이야?", "input": "", "output": "사과는 보통 빨간색이나 초록색이에요."},
  {"instruction": "2 더하기 2는?", "input": "", "output": "2 더하기 2는 4입니다."},
  {"instruction": "오늘 무슨 요일이야?", "input": "", "output": "오늘은 아마도 월요일일 거예요. 확인해볼게요!"},
  {"instruction": "좋아하는 색은?", "input": "", "output": "저는 인공지능이라 색을 느끼진 못하지만 파란색이 좋아 보여요."},
  {"instruction": "무슨 음악 좋아해?", "input": "", "output": "요즘은 클래식 음악이 인기가 많아요."},
  {"instruction": "영화 추천해줘", "input": "", "output": "최근에는 '듄'이나 '서울의 봄'이 인기 있어요."},
  {"instruction": "고양이와 강아지 중 누가 더 귀여워?", "input": "", "output": "둘 다 너무 귀엽죠! 사람마다 달라요."},
  {"instruction": "자기소개 해줘", "input": "", "output": "안녕하세요. 저는 여러분과 대화하는 인공지능 챗봇이에요."},
  {"instruction": "지금 몇 시야?", "input": "", "output": "제가 시간을 직접 알 순 없지만, 컴퓨터 시계를 확인해보세요."},
  {"instruction": "아침 먹었어?", "input": "", "output": "저는 음식을 먹지 않지만, 사용자님은 꼭 챙겨 드세요!"},
  {"instruction": "한국의 전통 음식은 뭐야?", "input": "", "output": "김치, 불고기, 비빔밥이 대표적인 전통 음식이에요."},
  {"instruction": "서울의 명소 알려줘", "input": "", "output": "경복궁, 남산타워, 한강공원이 유명해요."},
  {"instruction": "운동 추천해줘", "input": "", "output": "요가나 스트레칭은 부담 없이 시작할 수 있어요."},
  {"instruction": "책 추천해줘", "input": "", "output": "『아몬드』나 『지적 대화를 위한 넓고 얕은 지식』 어때요?"},
  {"instruction": "한국어 공부 어떻게 해?", "input": "", "output": "매일 조금씩 듣고 말하고 쓰는 연습이 좋아요."},
  {"instruction": "너무 피곤해", "input": "", "output": "그럴 땐 잠시 눈을 감고 쉬어보세요. 힘내요!"}
]

dataset = Dataset.from_list(examples)

#5. 전처리 (최대 토큰 32)
def preprocess(example):
  prompt = f"질문: {example['instruction']} {example['input']}\n답변:"
  inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=32)
  labels = tokenizer(example["output"], padding="max_length", truncation=True, max_length=32)
  inputs["labels"] = labels["input_ids"]
  return inputs

tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)

#6. 훈련 설정 (배치 1, 에폭 1, 로깅 최소화)
training_args = TrainingArguments(
  output_dir="./kogpt2-ultralight",
  per_device_train_batch_size=1,
  num_train_epochs=100,
  learning_rate=5e-5,
  save_steps=5,
  logging_dir="./logs",
  logging_steps=1,
  save_total_limit=1,
  report_to="none"
)

#7. Trainer 학습
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_dataset,
  tokenizer=tokenizer
)

trainer.train()

#8. 모델 저장
trainer.save_model("./kogpt2-ultralight")
tokenizer.save_pretrained("./kogpt2-ultralight")
print("초경량 파인튜닝 완료")

#9. 챗봇 생성
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_dir = "kogpt2-ultralight"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)


Map:   0%|          | 0/21 [00:00<?, ? examples/s]

/tmp/ipython-input-812696765.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 1}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,14.386000
2,13.172200
3,8.710300
4,5.183200
5,5.590000
6,6.785300
7,2.932600
8,2.389800
9,1.415000
10,5.059500


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

초경량 파인튜닝 완료


Device set to use cpu


In [ ]:
print("초경량 챗봇을 시작합니다. '종료'라고 입력하면 끝납니다.\n")
while True:
  user_input = input("사용자: ")
  if user_input.strip().lower() == "종료":
    break

  prompt = f"질문: {user_input}\n답변:"
  response = chatbot(prompt, max_new_tokens=50, do_sample=True, temperature=0.8)[0]["generated_text"]
  #response = chatbot(prompt, max_new_tokens=50, truncation=True, do_sample=True, temperature=0.8)[0]["generated_text"]
  answer = response.split("답변:")[-1].strip() if "답변:" in response else response.strip()
  print("🤖 챗봇:", answer)

초경량 챗봇을 시작합니다. '종료'라고 입력하면 끝납니다.

사용자: 뭐냐
🤖 챗봇: 요.
